In [1]:
# Notebook makes finds distribution of datasets after undersampling 1:3, before train/test split

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [4]:
all_assay_all_compounds =pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
all_assay_all_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,True,False,0.158783,0.500219
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,False,False,0.508641,0.530402
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,True,True,0.262399,0.319723
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,False,True,0.100980,0.628169
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,False,True,0.527003,0.289754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False,0.727044,0.698855
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True,0.214095,0.398324
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True,0.221627,0.651916
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True,0.163395,0.450736


In [5]:
len(all_assay_all_compounds.assay.unique())

88

In [6]:
stats_list=[]
for assay in all_assay_all_compounds.assay.unique():   
    per_assay_compounds=all_assay_all_compounds[all_assay_all_compounds["assay"]==assay]
    inactives= per_assay_compounds["true"].value_counts()[0]
    actives= per_assay_compounds["true"].value_counts()[1]
    total=inactives+actives
    
    row=[assay, total, actives, inactives]
    stats_list.append(row)

In [7]:
df = pd.DataFrame(stats_list, columns=["assay", "total", "actives", "inactives"])
df["ratio"] = df["actives"]/df["total"]
df

,assay,total,actives,inactives,ratio
0,588458,448,112,336,0.250000
1,588334,528,132,396,0.250000
2,2642,764,191,573,0.250000
3,2156,568,142,426,0.250000
4,2330,328,82,246,0.250000
...,...,...,...,...,...
83,720504,128,96,32,0.750000
84,720532,301,210,91,0.697674
85,1159524,880,220,660,0.250000
86,1117304,386,126,260,0.326425


In [8]:
df.describe()

,total,actives,inactives,ratio
count,88.000000,88.000000,88.000000,88.000000
mean,570.409091,197.193182,373.215909,0.369599
std,560.809055,203.353995,406.637669,0.173943
min,100.000000,25.000000,28.000000,0.250000
25%,264.750000,79.500000,109.500000,0.250000
50%,436.000000,133.500000,298.500000,0.250000
75%,714.500000,229.000000,449.250000,0.407937
max,3416.000000,1224.000000,2562.000000,0.750000


In [9]:
df.to_csv("assay_compounds_stats_list_bothsideundersampling.csv", index=False)